# gensim을 이용한 FastText

### 데이터 다운로드

https://github.com/e9t/nsmc/raw/master/ratings_train.txt 를 다운로드 받아 저장 후 연다.

In [1]:
import requests

res = requests.get('https://github.com/e9t/nsmc/raw/master/ratings_train.txt')

with open('../data/ratings_train.txt', 'wb') as f:
    f.write(res.content)

In [2]:
import pandas as pd

nsmc = pd.read_csv('../data/ratings_train.txt', sep='\t')

In [3]:
nsmc.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


### 전처리

In [4]:
import re

In [5]:
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text) # 한글자이상의 한글을 모두 찾기

In [6]:
find_hangul(nsmc.loc[0,'document'])

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [7]:
nsmc[nsmc['document'].notnull()] # 5개의 비어있는 행을 제외

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [8]:
data = nsmc[nsmc['document'].notnull()]['document'].map(find_hangul)

In [9]:
data[0]

['아', '더빙', '진짜', '짜증나네요', '목소리']

파일로 저장하는 방법도 있다. 먼저, 한글이 아닌 글자를 지우고 공백을 하나로 합친다.

In [10]:
def only_hangul(text):
    return ' '.join(find_hangul(text))

In [11]:
only_hangul(nsmc.loc[0,'document'])

'아 더빙 진짜 짜증나네요 목소리'

In [12]:
data2 = nsmc[nsmc['document'].notnull()]['document'].map(only_hangul)

In [13]:
data2[0]

'아 더빙 진짜 짜증나네요 목소리'

In [14]:
with open('../data/nsmc.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(data2))

### FastText 모형 학습

In [15]:
from gensim.models.fasttext import FastText

In [16]:
from gensim.models.word2vec import Word2Vec

FastText 모형을 만든다. 설정값에는 다음과 같은 것들이 있다.

- size: 임베딩의 크기 (기본값 100)
- sg: 0이면 CBOW(기본값), 1이면 Skip-gram
- alpha: 학습률 (기본값 0.025)
- min_alpha: 최소 학습률. FastText는 학습과정에서 학습률을 이 수준까지 점점 낮춘다. (기본값 0.0001)
- window: 문장 내에서 주변 단어와 대상 단어의 최대 거리(기본값 5) 좌우로 몇개의 단어까지
- min_count: 임베딩을 학습할 단어의 최소 출현 빈도 (기본값 5)

Word2Vec도 사용방법은 같다.

In [19]:
model = FastText(size=16) # 단어하나마다 16차원으로 학습

어휘를 파악한다. 파일로 저장한 경우에는 sentence=data 대신에 corpus_file='nsmc.txt'라고 한다.

In [20]:
model.build_vocab(sentences=data)

In [21]:
# model.build_vocab(corpus_file='nsmc.txt')

모형을 학습시킨다.

In [22]:
model.train(
    sentences=data,
    epochs=5,
    total_examples=model.corpus_count,
    total_words=model.corpus_total_words
)

### 저장과 불러오기

저장

In [23]:
model.save('../data/nsmc.fasttext')

불러오기

In [24]:
model = FastText.load('../data/nsmc.fasttext')

# FastText 임베딩

### 모형 불러오기

In [25]:
from gensim.models.fasttext import FastText
model = FastText.load('../data/nsmc.fasttext')

### 단어 임베딩

'히어로'는 단어 임베딩이 학습되어 있다.

In [26]:
'히어로' in model.wv.vocab

True

In [27]:
model.wv['히어로']

array([-0.07591183,  0.48751894,  0.45523566,  0.7433716 , -0.52968186,
       -0.14514065, -0.7991363 ,  0.2994704 ,  0.23659381,  0.3168685 ,
       -0.7486586 , -0.03397258, -0.21045317, -0.2904963 ,  0.640995  ,
       -0.73162127], dtype=float32)

'슈퍼히어로'는 단어 임베딩이 없지만

In [28]:
'슈퍼히어로' in model.wv.vocab

False

준단어 토큰의 임베딩을 더해서 임베딩을 계산해준다.

In [29]:
model.wv['슈퍼히어로']

array([-0.05226331,  0.25863388,  0.22525036,  0.27956298, -0.21986212,
       -0.07017983, -0.2935896 ,  0.08455249,  0.1033233 ,  0.06964119,
       -0.33938393, -0.01331062, -0.08984356, -0.09376252,  0.27367273,
       -0.2665792 ], dtype=float32)

### 유사도

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

'히어로'와 '슈퍼히어로'의 유사도는 높다.

In [31]:
model.wv.similarity('슈퍼히어로', '히어로')

0.98800856

'히어로'와 '평론가'의 유사도는 상대적으로 낮다.

In [32]:
model.wv.similarity('히어로', '평론가')

0.68076

'평론가'와 비슷한 단어들

In [33]:
model.wv.most_similar('평론가')

[('점이상은', 0.9908885955810547),
 ('평론가들', 0.9907047748565674),
 ('점이라', 0.9893724322319031),
 ('평론', 0.9891233444213867),
 ('점이야', 0.9885176420211792),
 ('점대면', 0.9881371855735779),
 ('점대라', 0.988065779209137),
 ('점이라니', 0.9879131317138672),
 ('조정', 0.9878653883934021),
 ('점대지', 0.9877920150756836)]

# FastText를 이용한 감성분석

### 준비

학습된 FastText 모형을 불러온다.

In [34]:
from gensim.models.fasttext import FastText
ft = FastText.load('../data/nsmc.fasttext')

데이터를 불러온다.

In [35]:
import pandas as pd
nsmc = pd.read_csv('../data/ratings_train.txt', sep='\t')

### 전처리

리뷰가 있는 데이터만 선택한다.

In [36]:
df = nsmc[nsmc['document'].notnull()]

훈련용 데이터와 테스트용 데이터를 분할한다.

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
doc_train, doc_test, y_train, y_test = train_test_split(df['document'], df['label'], test_size=0.2, random_state=42)



한글 단어만 추출하는 함수를 만든다.

In [39]:
import re
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text)

1000, 16 크기의 행렬을 만든다.

In [40]:
import numpy as np
x_train = np.zeros((1000, 16))

각 문서에서 한글 단어를 찾아 단어 임베딩을 구하고, 이를 문서마다 평균을 낸다.

In [41]:
for i, doc in enumerate(doc_train.iloc[:1000]):
    vs = [ft.wv[word] for word in find_hangul(doc)]
    if vs:
        x_train[i,] = np.mean(vs, axis=0)

x_train은 각 문서의 단어 임베딩 평균이다.

In [42]:
x_train[0]

array([ 0.46497652,  1.30914772,  2.05310345,  0.70937788, -0.22061728,
        0.12521037, -0.55661809, -0.49261209,  0.84338641, -0.91445994,
       -1.5739845 , -0.66046977,  0.17000785, -0.60663056, -0.24857529,
       -0.97864735])

# 모형 학습

각 문서의 단어 임베딩 평균을 이용하여 감성을 예측하는 모형을 만든다.

In [43]:
import tensorflow as tf

In [44]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [45]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
model.fit(x_train, y_train.values[:1000], epochs=1)

32/32 [==============================] - 0s 1ms/step - loss: 0.8647 - accuracy: 0.4380


# 순환신경망 감성분석

### 데이터 준비

In [47]:
import pandas as pd
df = pd.read_csv('../data/imdb.zip')

In [48]:
import joblib
tk = joblib.load('../data/tokenizer.pkl')

데이터를 분할한다.

In [49]:
from sklearn.model_selection import train_test_split

In [50]:
review_train, review_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)




토큰화한다.

In [51]:
seqs = tk.texts_to_sequences(review_train)

### 순방향 순환신경망

In [52]:
import tensorflow as tf

패딩을 한다. 길이가 짧으면 앞쪽에 0을 채운다(padding='pre'). maxlen은 최대 길이를 지정할 수 있다. 지정하지 않으면 가장 긴 문자열의 길이로 지정된다. truncating='pre'는 maxlen보다 긴 문자열일 경우 앞쪽을 자른다. 뒤쪽을 자르게 하려면 'post'로 설정한다.

In [53]:
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs, maxlen=None, padding='pre', truncating='pre')




In [54]:
NUM_WORDS= tk.num_words + 1

Embedding에서 mask_zero=True로 설정하면 0으로 패딩된 부분의 예측은 손실에 반영하지 않는다.

In [55]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.LSTM(8),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [56]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 8)           16008     
_________________________________________________________________
lstm (LSTM)                  (None, 8)                 544       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 16,561
Trainable params: 16,561
Non-trainable params: 0
_________________________________________________________________


모형을 학습시킨다.

In [57]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [58]:
model.fit(pads, y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 2s 15ms/step - loss: 0.6932 - accuracy: 0.5200
Epoch 2/10
25/25 [==============================] - 0s 15ms/step - loss: 0.6911 - accuracy: 0.6438
Epoch 3/10
25/25 [==============================] - 0s 16ms/step - loss: 0.6865 - accuracy: 0.7050
Epoch 4/10
25/25 [==============================] - 0s 12ms/step - loss: 0.6733 - accuracy: 0.8338
Epoch 5/10
25/25 [==============================] - 0s 12ms/step - loss: 0.6213 - accuracy: 0.8475
Epoch 6/10
25/25 [==============================] - 0s 14ms/step - loss: 0.5146 - accuracy: 0.8637
Epoch 7/10
25/25 [==============================] - 0s 12ms/step - loss: 0.4312 - accuracy: 0.8913
Epoch 8/10
25/25 [==============================] - 0s 12ms/step - loss: 0.3727 - accuracy: 0.9162
Epoch 9/10
25/25 [==============================] - 0s 16ms/step - loss: 0.3169 - accuracy: 0.9425
Epoch 10/10
25/25 [==============================] - 0s 12ms/step - loss: 0.2830 - accuracy: 0.9425


### 역방향 순환신경망

패딩을 한다. 길이가 짧으면 뒤쪽에 0을 채운다(padding='post')

In [59]:
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs, padding='post')

In [60]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.LSTM(8, go_backwards=True),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

### 양방향 순환신경망

LSTM을 Bidirectional로 감싸주면 자동으로 순방향과 역방향 레이어를 넣어준다.

In [61]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [62]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 8)           16008     
_________________________________________________________________
bidirectional (Bidirectional (None, 16)                1088      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 17,113
Trainable params: 17,113
Non-trainable params: 0
_________________________________________________________________
